In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### Download the data and load it to Pandas. 

You can find them [here](https://drive.google.com/file/d/1NY6cmF9Shjw-dD7BD6bNmfcIVz-kQcFR/view?usp=sharing).

In [2]:
cast = pd.read_csv('data/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


In [3]:
release_dates = pd.read_csv('data/release_dates.csv', index_col=None,
                                      parse_dates=['date'], infer_datetime_format=True)
release_dates.head()

,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### Count the number of movies with "Christmas" in their title for each month, that is released are the USA.

In [4]:
# determine dtypes to help define new filter function
release_dates.dtypes

title              object
year                int64
country            object
date       datetime64[ns]
dtype: object

In [5]:
# filter_results function
# chooses return value based on the value type of the filtered field
def filter_results(table, field, value):
    if table[field].dtype == 'int64':
        return table[table[field] == value]
    else:
        return table[table[field].str.contains(value)]

In [6]:
release_dates.pipe(filter_results, 'title', 'Christmas').pipe(filter_results, 'country', 'USA')

,title,year,country,date
1191,12 Dog Days of Christmas,2014,USA,2014-11-28
1192,12 Dogs of Christmas: Great Puppy Rescue,2012,USA,2012-10-09
5904,A Belle for Christmas,2014,USA,2014-11-04
6163,A Cadaver Christmas,2011,USA,2011-04-02
6272,A Christmas Carol,1938,USA,1938-12-16
...,...,...,...,...
383807,The Nightmare Before Christmas,1993,USA,1993-10-29
395219,The Shootin' It Christmas Spectacular,2013,USA,2013-12-20
408785,This Christmas,2007,USA,2007-11-21
437275,What She Wants for Christmas,2012,USA,2012-12-01


### Count the number of movies with "Christmas" in their title for each month, that are released in Canada.

In [7]:
# determine date type
print(release_dates['date'].dtype.name)

datetime64[ns]


In [8]:
# determine min/max date values to search through
print(release_dates['date'].sort_values())

232584   1894-10-09
317627   1900-09-13
398115   1906-12-26
290219   1907-11-02
21527    1908-09-12
            ...    
365289   2021-12-07
74969    2021-12-24
284063   2022-03-25
312128   2022-09-01
165532   2023-03-01
Name: date, Length: 452656, dtype: datetime64[ns]


In [9]:
# create a monthly date range spanning the entire dated history of the release_date tables
bins = pd.date_range('1894-10', '2023-03', freq='MS')
canada_christmas_releases = release_dates.pipe(filter_results, 'title', 'Christmas').pipe(filter_results, 'country', 'Canada')
canada_christmas_releases.groupby(pd.cut(canada_christmas_releases['date'], bins))['title'].count().sort_values()

date
(1894-10-01, 1894-11-01]    0
(1980-02-01, 1980-03-01]    0
(1980-01-01, 1980-02-01]    0
(1979-12-01, 1980-01-01]    0
(1979-11-01, 1979-12-01]    0
                           ..
(2012-10-01, 2012-11-01]    1
(2011-11-01, 2011-12-01]    2
(2014-12-01, 2015-01-01]    2
(2014-11-01, 2014-12-01]    2
(2016-12-01, 2017-01-01]    3
Name: title, Length: 1541, dtype: int64

### Count the number of movies in which the titles start with "The Hobbit" for each month, that are released in the USA.

In [10]:
hobbit_usa_releases = release_dates.pipe(filter_results, 'title', 'The Hobbit').pipe(filter_results, 'country', 'USA')
hobbit_usa_releases.groupby(pd.cut(release_dates['date'], bins))['title'].count().sort_values()

date
(1894-10-01, 1894-11-01]    0
(1980-08-01, 1980-09-01]    0
(1980-07-01, 1980-08-01]    0
(1980-06-01, 1980-07-01]    0
(1980-05-01, 1980-06-01]    0
                           ..
(1937-10-01, 1937-11-01]    0
(1935-02-01, 1935-03-01]    0
(2014-12-01, 2015-01-01]    1
(2013-12-01, 2014-01-01]    1
(2012-12-01, 2013-01-01]    1
Name: title, Length: 1541, dtype: int64

### Count the number of movies with "Romance" in their title for **each day of the week**, that are released in the USA.

In [11]:
def which_day(year_start, year_end, day):
    
    annual_list = []
    
    for year in range(year_start, year_end):
        
        if day == 'Monday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-MON').strftime('%m/%d/%Y').to_list())
            
        elif day == 'Tuesday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-TUE').strftime('%m/%d/%Y').to_list())
            
        elif day == 'Wednesday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-WED').strftime('%m/%d/%Y').to_list())
            
        elif day == 'Thursday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-THU').strftime('%m/%d/%Y').to_list())
            
        elif day == 'Friday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-FRI').strftime('%m/%d/%Y').to_list())
            
        elif day == 'Saturday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W-SAT').strftime('%m/%d/%Y').to_list())
        
        elif day == 'Sunday':
            annual_list.extend(
                pd.date_range(
                    start=str(year),
                    end=str(year+1),
                    freq='W').strftime('%m/%d/%Y').to_list())
        
        else:
            print('No such day exists.')
            return None
        
    return annual_list

In [13]:
romance_titles = release_dates.pipe(filter_results, 'title', 'Romance')
romance_titles.sort_values('date')

,title,year,country,date
5988,A Blue Gum Romance,1913,Australia,1913-09-20
411457,Tillie's Punctured Romance,1914,USA,1914-12-21
448635,Young Romance,1915,USA,1915-01-21
343033,The Bachelor's Romance,1915,USA,1915-02-11
392256,The Romance of Elaine,1915,USA,1915-06-14
...,...,...,...,...
222409,Manhattan Romance,2015,USA,2015-10-02
292005,Romance Out of the Blue,2015,China,2015-10-23
291998,Romance Complicated,2016,India,2016-01-15
21168,American Romance,2016,USA,2016-10-23


In [14]:
for weekday in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
    days = which_day(1894, 2024, weekday)
    day_count = romance_titles[romance_titles['date'].isin(days)].sort_values('date')['date'].count()
    print(f'{weekday} : {day_count}')

Sunday : 45
Monday : 65
Tuesday : 16
Wednesday : 34
Thursday : 53
Friday : 106
Saturday : 21


### Count the number of movies with "Action" in their title for **each day of the week**, that are released in the USA.

In [15]:
action_titles = release_dates.pipe(filter_results, 'title', 'Action')
action_titles.sort_values('date')

,title,year,country,date
13441,Action,1921,USA,1921-09-12
8462,A Man of Action,1923,USA,1923-06-03
13442,Action,1921,France,1924-01-11
8463,A Man of Action,1923,Denmark,1925-01-05
13443,Action,1921,Finland,1925-02-15
...,...,...,...,...
13451,Action Hero Biju,2016,United Arab Emirates,2016-02-25
13450,Action Hero Biju,2016,Kuwait,2016-02-25
426049,Untitled Disney Live-Action Fairy Tale,2017,USA,2017-12-22
426050,Untitled Disney Live-Action Fairy Tale,2019,USA,2019-03-29


In [16]:
for weekday in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
    days = which_day(1894, 2024, weekday)
    day_count = action_titles[action_titles['date'].isin(days)].sort_values('date')['date'].count()
    print(f'{weekday} : {day_count}')

Sunday : 9
Monday : 16
Tuesday : 2
Wednesday : 36
Thursday : 54
Friday : 115
Saturday : 19


### On which date was each Judi Dench movie from the 1990s released in the USA?

In [39]:
judi_dench_titles = cast.pipe(filter_results, 'name', 'Judi Dench')
judi_dench_titles = judi_dench_titles[judi_dench_titles['year'].isin(range(1990, 2000))]
judi_dench_titles

,title,year,name,type,character,n
2720483,GoldenEye,1995,Judi Dench,actress,M,6.0
2720484,Hamlet,1996,Judi Dench,actress,Hecuba,12.0
2720490,Jack & Sarah,1995,Judi Dench,actress,Margaret,3.0
2720495,Mrs Brown,1997,Judi Dench,actress,Queen Victoria,1.0
2720508,Shakespeare in Love,1998,Judi Dench,actress,Queen Elizabeth,12.0
2720512,Tea with Mussolini,1999,Judi Dench,actress,Arabella,2.0
2720520,The World Is Not Enough,1999,Judi Dench,actress,M,6.0
2720521,Tomorrow Never Dies,1997,Judi Dench,actress,M,9.0


In [43]:
usa_release_dates = release_dates.pipe(filter_results, 'country', 'USA')
usa_release_dates[(usa_release_dates['title'].isin(judi_dench_titles['title'])) & (usa_release_dates['year'].isin(judi_dench_titles['year']))]

,title,year,country,date
132332,GoldenEye,1995,USA,1995-11-17
138986,Hamlet,1996,USA,1996-12-25
169109,Jack & Sarah,1995,USA,1996-03-22
239011,Mrs Brown,1997,USA,1997-10-03
306506,Shakespeare in Love,1998,USA,1999-01-08
336559,Tea with Mussolini,1999,USA,1999-05-14
406606,The World Is Not Enough,1999,USA,1999-11-19
413573,Tomorrow Never Dies,1997,USA,1997-12-19


### In which months do films with the actress Judi Dench tend to be released in the USA?

In [57]:
judi_dench = cast.pipe(filter_results, 'name', 'Judi Dench')

judi_dench_release_dates = release_dates[
    (release_dates['title'].isin(judi_dench['title'])) & (release_dates['year'].isin(judi_dench['year']))]

In [58]:
judi_dench_release_dates

,title,year,country,date
5499,84 Charing Cross Road,1987,USA,1987-02-13
5500,84 Charing Cross Road,1987,Australia,1987-04-30
5501,84 Charing Cross Road,1987,Ireland,1987-05-29
5502,84 Charing Cross Road,1987,Netherlands,1987-11-12
5503,84 Charing Cross Road,1987,Brazil,1987-12-25
...,...,...,...,...
436882,Wetherby,1985,USA,1985-07-19
436883,Wetherby,1985,West Germany,1985-11-21
436884,Wetherby,1985,Denmark,1985-12-06
436885,Wetherby,1985,Netherlands,1986-05-02


In [71]:
judi_dench_release_dates.pipe(filter_results, 'country', 'USA').groupby(judi_dench_release_dates['date'].dt.month).agg('count')['title']

date
1      5
2      4
3      3
4      3
5      5
6      5
7      1
8      1
9      2
10     1
11    13
12     4
Name: title, dtype: int64

### In which months do films with the actor Tom Cruise tend to be released in the USA?

In [67]:
tom_cruise_cast = cast.pipe(filter_results, 'name', 'Tom Cruise')
tom_cruise_release_dates = release_dates[(release_dates['title'].isin(tom_cruise_cast['title'])) & (release_dates['country'] == 'USA')]

In [68]:
tom_cruise_release_dates

,title,year,country,date
7003,A Few Good Men,1992,USA,1992-12-11
18669,All the Right Moves,1983,USA,1983-10-21
20884,American Made,2017,USA,2017-09-29
30382,August,1996,USA,1996-04-19
30592,Austin Powers in Goldmember,2002,USA,2002-07-26
49388,Born on the Fourth of July,1989,USA,1990-01-05
68953,Cocktail,1988,USA,1988-07-29
69501,Collateral,2004,USA,2004-08-06
81281,Days of Thunder,1990,USA,1990-06-27
100931,Edge of Tomorrow,2014,USA,2014-06-06


In [70]:
tom_cruise_release_dates.groupby(tom_cruise_release_dates['date'].dt.month).count()['title']

date
1      2
2      2
3      2
4      4
5      6
6      8
7      6
8      3
9      1
10     4
11     4
12    12
Name: title, dtype: int64